In [1]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, default_data_collator, \
    DataCollatorForLanguageModeling
from datasets import load_dataset
from arguments import DataTrainingArguments, ModelArguments, TrainingArguments

# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base', legacy=True)

# Initialize the data arguments
data_args = DataTrainingArguments(
    datasets='floorplan',
    data_dir=None,
    train_split='train',
    max_seq_length=512,
    max_output_seq_length=512,
    max_seq_length_eval=512,
    max_output_seq_length_eval=512,
    exp='full',
    boundary_in_where='Encoder'
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
model_args = ModelArguments(model_name_or_path='t5-base', config_name=None, tokenizer_name=None, cache_dir=None)

training_args = TrainingArguments()

In [3]:
training_args.do_train

False

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    padding="max_length",
)

In [5]:
dataset = load_dataset('floorplan', data_args, split='train',
                       max_input_length=data_args.max_seq_length, max_output_length=data_args.max_output_seq_length,
                       tokenizer=tokenizer, seed=1, train_subset=data_args.train_subset)

Example input sententece: Balcony 1 is about 8 foot X 2 foot . This is located right off of the kitchen . Balcony 2 is about 10 foot X 3 foot . This is facing north of the house . The bathroom is about 8 foot X 6 foot . It is located between the living room and the kitchen . Common room 1 is about 10 foot X 12 foot . The common rooms are located diagonal from each other . Common room 2 is about 10 foot X 10 foot . The common rooms are located diagonal from each other . The kitchen is a 10 foot X 10 foot room . It looks like it is opened up to the living room . The living room is located on the east side of the house . It is shaped like a T and it's the biggest room . It is approximately 14 feet X 32 feet . Each room that you come out of you will enter the living room . The master is about 10 feet X 15 feet . It is located far west of the house . It is a rectangle shape .+ 43 51 213 205 - 196 202 218 180 - 175 79 202 52 - 170 122 224 68 - 132 83 165 50 - 42 115 106 50 - 42 203 57 188
Ex

AttributeError: 'list' object has no attribute 'size'

In [26]:
model = T5ForConditionalGeneration.from_pretrained("t5-base")

In [27]:
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, mlm_probability=0.15)

In [43]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    # data_collator=default_data_collator,
)

/Users/thebui/Developer/projects/t2f/venv/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [44]:
trainer.train()

TypeError: vars() argument must have __dict__ attribute

In [6]:
import json

file_path = 'data/floorplan/floorplan_train.json'

with open(file_path, 'r') as f:
    data = json.load(f)

In [10]:
data[0]['rooms']

[{'room_type': 'master room',
  'x': 102,
  'y': 77,
  'h': 59,
  'w': 42,
  'x_min': 73,
  'y_min': 56,
  'x_max': 131,
  'y_max': 97,
  'near_x_min': -1,
  'near_y_min': -1,
  'near_x_max': -1,
  'near_y_max': -1,
  'relation': [],
  'location': 'south west corner',
  'size': '150',
  'aspect ratio': '5 over 7',
  'private': False},
 {'room_type': 'common room 1',
  'x': 154,
  'y': 96,
  'h': 34,
  'w': 47,
  'x_min': 137,
  'y_min': 73,
  'x_max': 170,
  'y_max': 119,
  'near_x_min': -1,
  'near_y_min': -1,
  'near_x_max': -1,
  'near_y_max': -1,
  'relation': [],
  'location': 'north west corner',
  'size': '100',
  'aspect ratio': '11 over 8',
  'private': False},
 {'room_type': 'balcony 1',
  'x': 192,
  'y': 139,
  'h': 33,
  'w': 12,
  'x_min': 176,
  'y_min': 133,
  'x_max': 208,
  'y_max': 144,
  'near_x_min': -1,
  'near_y_min': -1,
  'near_x_max': -1,
  'near_y_max': -1,
  'relation': [],
  'location': 'north side',
  'size': '25',
  'aspect ratio': '4 over 11',
  'private

In [67]:
from datasets import FloorplanDataset

floorplan_dataset = FloorplanDataset(tokenizer=tokenizer, max_input_length=512, max_output_length=512, data_args=data_args, seed=1)

In [65]:
len(floorplan_dataset.features)

2246

In [60]:
type(floorplan_dataset)

datasets.FloorplanDataset

In [16]:
data_args.output_format_type

'short'

In [68]:
example = floorplan_dataset.features[0]
print(example.keys())

dict_keys(['encoder_input_ids', 'encoder_attention_mask', 'encoder_num_rooms', 'encoder_reg_labels', 'decoder_boundary_ids', 'decoder_boundary_mask', 'decoder_labels'])


In [54]:
from torch.utils.data import Dataset, DataLoader


class MyDataset(Dataset):
    def __init__(self, features):
        self.features = features

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        feature_dict = {
            'input_ids': feature.input_ids,
            'attention_mask': feature.attention_mask,
            'labels': feature.label_ids,
            # 'boundary_ids': feature.boundary_ids,
            # 'boundary_mask': feature.boundary_mask,
            'num_rooms': feature.num_rooms,
            'regr_labels': feature.regr_labels,
            # 'decoder_attention_mask': feature.decoder_attention_mask,
        }
        for key, value in feature_dict.items():
            if value is None:
                print(f"Feature '{key}' is None for index {idx}")
        return feature_dict


dataset = MyDataset(floorplan_dataset.features)

In [55]:
len(dataset)

2246

In [56]:
for i in range(len(dataset)):
    dataset.__getitem__(i)

In [62]:
dataloader = DataLoader(floorplan_dataset, batch_size=32)

for batch in dataloader:
    print(batch)
    break

{'encoder_input_ids': [tensor([37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37]), tensor([2582, 2582, 2582, 2582, 2582, 2582, 2582, 2582, 2582, 2582, 2582, 2582,
        2582, 2582, 2582, 2582, 2582, 2582, 2582, 2582, 2582, 2582, 2582, 2582,
        2582, 2582, 2582, 2582, 2582, 2582, 2582, 2582]), tensor([19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19,
        19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]), tensor([1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069,
        1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069,
        1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069]), tensor([2069, 2069, 2069, 2069, 2069, 2069, 2069, 2069, 2069, 2069, 2069, 2069,
        2069, 2069, 2069, 2069, 2069, 2069, 2069, 2069, 2069, 2069, 2069, 2069,
        2069, 2069, 2069, 2069, 2069, 2069, 2069, 2069]), tensor([120, 120, 120, 120, 120

In [63]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=floorplan_dataset,
    # data_collator=default_data_collator,
)
trainer.train()

/Users/thebui/Developer/projects/t2f/venv/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,decoder_input_ids,decoder_attention_mask,head_mask,decoder_head_mask,cross_attn_head_mask,encoder_outputs,past_key_values,inputs_embeds,decoder_inputs_embeds,labels,use_cache,output_attentions,output_hidden_states,return_dict,label_ids,labels,label.